In [1]:
"""
import ibmos2spark
# @hidden_cell
credentials = {
    'endpoint': 'https://s3.eu-geo.objectstorage.service.networklayer.com',
    'service_id': 'iam-ServiceId-1d71bac5-a341-449f-9ca7-b29de95e771a',
    'iam_service_endpoint': 'https://iam.eu-gb.bluemix.net/oidc/token',
    'api_key': 'ejujKveBRS3Bk7l3pzjemEQDhTSaOKmqCg6x6osXWkx4'
}

configuration_name = 'os_2d970470a6354234a1716fe9f4db519b_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

df = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('US_Accidents_Dec19.csv', 'bdppproject-donotdelete-pr-nejualq57kuqe4'))
"""

"\nimport ibmos2spark\n# @hidden_cell\ncredentials = {\n    'endpoint': 'https://s3.eu-geo.objectstorage.service.networklayer.com',\n    'service_id': 'iam-ServiceId-1d71bac5-a341-449f-9ca7-b29de95e771a',\n    'iam_service_endpoint': 'https://iam.eu-gb.bluemix.net/oidc/token',\n    'api_key': 'ejujKveBRS3Bk7l3pzjemEQDhTSaOKmqCg6x6osXWkx4'\n}\n\nconfiguration_name = 'os_2d970470a6354234a1716fe9f4db519b_configs'\ncos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')\n\nfrom pyspark.sql import SparkSession\nspark = SparkSession.builder.getOrCreate()\n\ndf = spark.read  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')  .option('header', 'true')  .load(cos.url('US_Accidents_Dec19.csv', 'bdppproject-donotdelete-pr-nejualq57kuqe4'))\n"

In [2]:
from functions import *
from datetime import datetime
import numpy as np

from pyspark.ml.feature import VectorAssembler,OneHotEncoderEstimator,QuantileDiscretizer, StringIndexer, Imputer,StandardScaler,MinMaxScaler
from pyspark.sql.functions import udf
from pyspark.ml.linalg import SparseVector, VectorUDT
from pyspark.ml import Pipeline
from pyspark.ml.feature import ChiSqSelector,PCA
from pyspark.storagelevel import StorageLevel

from pyspark.ml.classification import LogisticRegression,DecisionTreeClassifier,RandomForestClassifier
from pyspark.ml.tuning import ParamGridBuilder,CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [3]:
model_indexing_dir = "log_index.md"
models_dir = "models"
logs_dir = "models" 
filter_state = 'CA' # ALL if all states
model_Note = ''
log_mode=True # Logs data to files, Dont use on massive dataset (debug_mode is recommended while on)
debug_mode=False # Reduce number of rows collected for faster processing
enable_plots = True # Dont use unless debug_mode is on, or datasize is small
logger = logging(models_dir, logs_dir, "", filter_state, enabled=log_mode)

file = "data/US_Accidents_Dec19.csv"


df, sc,spark = setup_spark(file)

In [4]:
sc

<SparkContext master=local[*] appName=Spark Project>

In [5]:
tmp = sc._conf.getAll()
logger.write2file("New Spark session", str(tmp))
print("Config:",tmp)

Config: [('spark.memory.offHeap.size', '4g'), ('spark.driver.port', '44161'), ('spark.app.id', 'local-1589634946008'), ('spark.executor.memory', '4g'), ('spark.driver.host', '4082e07eadd9'), ('spark.ui.enabled', 'true'), ('spark.executor.id', 'driver'), ('spark.app.name', 'Spark Project'), ('spark.memory.offHeap.enabled', 'true'), ('spark.ui.killEnabled', 'false'), ('spark.serializer', 'org.apache.spark.serializer.KryoSerializer'), ('spark.driver.memory', '6g'), ('spark.rdd.compress', 'True'), ('spark.serializer.objectStreamReset', '100'), ('spark.executor.instances', '1'), ('spark.kryoserializer.buffer.max', '256m'), ('spark.master', 'local[*]'), ('spark.executor.cores', '1'), ('spark.submit.deployMode', 'client'), ('spark.driver.maxResultSize', '3g'), ('spark.ui.showConsoleProgress', 'true'), ('spark.driver.cores', '1')]


# Debug tool

In [6]:
if debug_mode == True: 
    df.createOrReplaceTempView("records")
    reviewData = spark.sql("SELECT * FROM records LIMIT 100000") 

df.count()

2925212

# 1. Define variables

In [7]:
colLabel = ["Severity"]

colRem = ['ID', 
          'Source',
          'End_Time',
          'End_Lat',
          'End_Lng',
          'Description',
        ]

df, colCat, colNum = setup_variables(df, sc, colLabel, colRem)  

logger.write2file("Number of rows", str(df.count()))
logger.write2file("Categorical groups","Defined Label:\n" + str(colLabel) + "\nDefined Categories:\n" + str(colCat) + "\nDefined Numerical:\n" +str(colNum))

Label: ['Severity'] 
Categories: ['Street', 'Side', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone', 'Airport_Code', 'Wind_Direction', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight']
Numerical: ['TMC', 'Start_Lat', 'Start_Lng', 'Distance(mi)', 'Number', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)', 'Precipitation(in)']


# 2. Preprocessing

## Analyse single state
#Create a model for separates states instead of all states at once.

In [8]:
if filter_state != 'ALL':
    df = df.filter(df.State == filter_state) # Lowers the dataset quite a lot
    logger.write2file("Specified state",str(filter_state))
else:
    logger.write2file("No state specified","")

## Modify time
#Convert the timestamp into a numeric value and then into a string so that the time of day and month can be categorised.

In [9]:
# Convert to int then cast to string

df = df.withColumn('Start_Hour', hour(to_timestamp(from_utc_timestamp(df['Start_Time'], df['Timezone']).cast('string'), 'yyyy-MM-dd HH:mm:ss')))
df = df.withColumn('Start_Month', month(to_timestamp(from_utc_timestamp(df['Start_Time'], df['Timezone']).cast('string'), 'yyyy-MM-dd HH:mm:ss')))

df = df.withColumn('Weather_Hour', hour(to_timestamp(from_utc_timestamp(df['Weather_Timestamp'], df['Timezone']).cast('string'), 'yyyy-MM-dd HH:mm:ss')))
df = df.withColumn('Weather_Month', month(to_timestamp(from_utc_timestamp(df['Weather_Timestamp'], df['Timezone']).cast('string'), 'yyyy-MM-dd HH:mm:ss')))

df.show(1)
logger.write2file("Dataset after modifying UTC timestamp", str(df.take(1)))

+-----+--------+-------------------+---------+-----------+------------+------+-----------+----+-------+------+-----+-------+-------+----------+------------+-------------------+--------------+-------------+-----------+------------+--------------+--------------+---------------+-----------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+--------------+--------------+-----------------+---------------------+----------+-----------+------------+-------------+
|  TMC|Severity|         Start_Time|Start_Lat|  Start_Lng|Distance(mi)|Number|     Street|Side|   City|County|State|Zipcode|Country|  Timezone|Airport_Code|  Weather_Timestamp|Temperature(F)|Wind_Chill(F)|Humidity(%)|Pressure(in)|Visibility(mi)|Wind_Direction|Wind_Speed(mph)|Precipitation(in)|Weather_Condition|Amenity| Bump|Crossing|Give_Way|Junction|No_Exit|Railway|Roundabout|Station| Stop|Traffic_Calming|Traffic_Signal|Turning_Loop|Sun

## Quantiles function
#Alternative solution to get position 
#Remove City, Country, State, Zipcode, Airport_Code
#In order to reduce dimensionality feature crossing will be applied upon the Start_Lat and Start_Lng then fused in order to create a new set of data

In [10]:
use_feature_cross = False
if use_feature_cross: # Depricated due to limited amount of hardware (not enough performance in RAM) to do a correct feature cross!
    def cross_func(x,y): 
        
        """
        Feature crossing of data
        """
        position = np.outer(np.array(x), np.array(y)).flatten()
        nonzero = np.nonzero(position)[0]
        return SparseVector(position.size, nonzero, position[nonzero])


    cross_udf = udf(cross_func, VectorUDT())

    # Calculate how many buckets needed (Bins)
    discretizer_Lat = QuantileDiscretizer(numBuckets=20, inputCol="Start_Lat", outputCol="Start_Lat_disc")
    discretizer_Lng = QuantileDiscretizer(numBuckets=20, inputCol="Start_Lng", outputCol="Start_Lng_disc")

    # Into categorical values
    indexer_cord = [StringIndexer(inputCol=c + "_disc", outputCol=c+"_IDX") for c in ["Start_Lat","Start_Lng"]]

    # One-hot (feature crossing)
    encoder_cord = OneHotEncoderEstimator(inputCols=[indexer.getOutputCol() for indexer in indexer_cord], outputCols=["{0}_vec".format(indexer.getOutputCol()) for indexer in indexer_cord])

    cord_stages = [discretizer_Lat,discretizer_Lng, *indexer_cord, encoder_cord]
    #pipeline_cord = Pipeline(stages=[])
    #preprocessed_cord = pipeline_cord.fit(df).transform(df)

    # Feature crossing - Since One-hot-encoding works very bad with this it is sadly disabled. It works but the time to calculate the vector is to great with One-hot encoding
    
    # Following row of code is used instead to give an example of how it would work.
    #position = VectorAssembler(inputCols=["Start_Lat_IDX_vec","Start_Lng_IDX_vec"], outputCol="position")
    #f = position.transform(preprocessed_cord)
    #preprocessed_cord.select('position').show(10)

## Cluster quantiles to reduce the feature vector for position
#Since feature crossing wont work due to the feature vector is to big (regarding to the given resources) we hope that Clustering might reduce them and keep as much data as possible

In [11]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

number_of_clusters= 400
pos_assembler = VectorAssembler(inputCols=['Start_Lat', 'Start_Lng'], outputCol="position_features")
preprocessed_cord = pos_assembler.transform(df)

trainSet, testSet = preprocessed_cord.randomSplit([0.9 ,0.1], 1)
kmeans = KMeans(k=number_of_clusters, featuresCol='position_features', predictionCol='position')
model = kmeans.fit(trainSet)
predictions = model.transform(testSet)

evaluator = ClusteringEvaluator(featuresCol='position_features', predictionCol='position')

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

# Transform the hole dataset
df = model.transform(preprocessed_cord)
df.select('position').show(5)

Silhouette with squared euclidean distance = 0.601691964398246
+--------+
|position|
+--------+
|     126|
|     128|
|     105|
|     240|
|       2|
+--------+
only showing top 5 rows



In [12]:
df.show(1)
logger.write2file("Dataset after adding quantiles",str(df.take(1)))

+-----+--------+-------------------+---------+-----------+------------+------+-----------+----+-------+------+-----+-------+-------+----------+------------+-------------------+--------------+-------------+-----------+------------+--------------+--------------+---------------+-----------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+--------------+--------------+-----------------+---------------------+----------+-----------+------------+-------------+--------------------+--------+
|  TMC|Severity|         Start_Time|Start_Lat|  Start_Lng|Distance(mi)|Number|     Street|Side|   City|County|State|Zipcode|Country|  Timezone|Airport_Code|  Weather_Timestamp|Temperature(F)|Wind_Chill(F)|Humidity(%)|Pressure(in)|Visibility(mi)|Wind_Direction|Wind_Speed(mph)|Precipitation(in)|Weather_Condition|Amenity| Bump|Crossing|Give_Way|Junction|No_Exit|Railway|Roundabout|Station| Stop|Traffic_Calming|T

## Cast data 
#Cast all the datacolumns into correct format so they will be sorted to numerical or categorical values. Data removed caused to big feature vector, contained too #many NaN values or were converted into another form.

In [14]:
# Could be handles as separated lists and then sent to a UDF for processing but since low number of features and easier to visualize this select is kept.
df = df.select(
        col('Severity').cast('int'),
        col('Start_Hour').cast('string'),
        col('Start_Month').cast('string'),
        col('Weather_Hour').cast('string'),
        col('Weather_Month').cast('string'),
        col('position').cast('string'),
        col('Distance(mi)').cast('double'),
        col('Side').cast('string'),   
        col('Temperature(F)').cast('double'),
        col('Wind_Chill(F)').cast('double'), 
        col('Humidity(%)').cast('double'),
        col('Pressure(in)').cast('double'),
        col('Visibility(mi)').cast('double'),
        col('Wind_Direction').cast('string'),
        col('Wind_Speed(mph)').cast('double'), 
        col('Weather_Condition').cast('string'),
        col('Amenity').cast('string'),
        col('Bump').cast('string'),
        col('Crossing').cast('string'),
        col('Give_Way').cast('string'),
        col('Junction').cast('string'),
        col('No_Exit').cast('string'),
        col('Railway').cast('string'),
        col('Roundabout').cast('string'),
        col('Station').cast('string'),
        col('Stop').cast('string'),
        col('Traffic_Calming').cast('string'),
        col('Traffic_Signal').cast('string'),
        col('Turning_Loop').cast('string'),
        col('Sunrise_Sunset').cast('string'),
        col('Civil_Twilight').cast('string'),
        col('Nautical_Twilight').cast('string'),
        col('Astronomical_Twilight').cast('string')
    ) 

colLabel = ["Severity"]

colCat, colNum = createNewClasses(df, sc, colLabel)
logger.write2file("Categorical groups","Defined Label:\n" + str(colLabel) + "\nDefined Categories:\n" + str(colCat) + "\nDefined Numerical:\n" +str(colNum))

Label: ['Severity'] 
Categories: ['Start_Hour', 'Start_Month', 'Weather_Hour', 'Weather_Month', 'position', 'Side', 'Wind_Direction', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight']
Numerical: ['Distance(mi)', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)']


## Get categorical values

In [15]:
df.persist()
tmp = [df.select(countDistinct(c).alias(c)).collect()[0] for c in [*colCat]] 
df.unpersist()
print("Unique column values:", tmp)

logger.write2file("Unique column values", str(tmp))

Unique column values: [Row(Start_Hour=24), Row(Start_Month=12), Row(Weather_Hour=24), Row(Weather_Month=12), Row(position=400), Row(Side=3), Row(Wind_Direction=24), Row(Weather_Condition=69), Row(Amenity=2), Row(Bump=2), Row(Crossing=2), Row(Give_Way=2), Row(Junction=2), Row(No_Exit=2), Row(Railway=2), Row(Roundabout=2), Row(Station=2), Row(Stop=2), Row(Traffic_Calming=2), Row(Traffic_Signal=2), Row(Turning_Loop=1), Row(Sunrise_Sunset=2), Row(Civil_Twilight=2), Row(Nautical_Twilight=2), Row(Astronomical_Twilight=2)]


## Clean data

### Recheck the missing values
#Check so that the output contains 0 missing values

In [16]:
df.persist(StorageLevel.MEMORY_AND_DISK)
printMissingValues(df,logger)
df.unpersist()

Missing valuesroot
 |-- Severity: long (nullable = false)
 |-- Start_Hour: long (nullable = false)
 |-- Start_Month: long (nullable = false)
 |-- Weather_Hour: long (nullable = false)
 |-- Weather_Month: long (nullable = false)
 |-- position: long (nullable = false)
 |-- Distance(mi): long (nullable = false)
 |-- Side: long (nullable = false)
 |-- Temperature(F): long (nullable = false)
 |-- Wind_Chill(F): long (nullable = false)
 |-- Humidity(%): long (nullable = false)
 |-- Pressure(in): long (nullable = false)
 |-- Visibility(mi): long (nullable = false)
 |-- Wind_Direction: long (nullable = false)
 |-- Wind_Speed(mph): long (nullable = false)
 |-- Weather_Condition: long (nullable = false)
 |-- Amenity: long (nullable = false)
 |-- Bump: long (nullable = false)
 |-- Crossing: long (nullable = false)
 |-- Give_Way: long (nullable = false)
 |-- Junction: long (nullable = false)
 |-- No_Exit: long (nullable = false)
 |-- Railway: long (nullable = false)
 |-- Roundabout: long (nullable

DataFrame[Severity: int, Start_Hour: string, Start_Month: string, Weather_Hour: string, Weather_Month: string, position: string, Distance(mi): double, Side: string, Temperature(F): double, Wind_Chill(F): double, Humidity(%): double, Pressure(in): double, Visibility(mi): double, Wind_Direction: string, Wind_Speed(mph): double, Weather_Condition: string, Amenity: string, Bump: string, Crossing: string, Give_Way: string, Junction: string, No_Exit: string, Railway: string, Roundabout: string, Station: string, Stop: string, Traffic_Calming: string, Traffic_Signal: string, Turning_Loop: string, Sunrise_Sunset: string, Civil_Twilight: string, Nautical_Twilight: string, Astronomical_Twilight: string]

### Remove data with occurance less than 1%
#Based on information from analysis. With further analysis lower procentage can be used to find better results.

In [17]:
df.persist()
original_rows = df.count()
n = int(df.count()*0.001) # Limit the plot to ignore conditions below an limit

weather_freq = df.groupBy('Weather_Condition').count().orderBy('count',ascending=False)
df_filtered = weather_freq.filter(weather_freq['count'] > n)
filtered_conditions = df_filtered.select("Weather_Condition").rdd.flatMap(lambda x: x).collect()
df = df.filter(df['Weather_Condition'].isin(*filtered_conditions))

df_filtered.show()
removed_rows = original_rows - df.count()

df.unpersist()
print("Rows removed:",removed_rows)

logger.write2file("Weather condition", str(df_filtered.take(df_filtered.count())) + "\nRows removed: " + str(removed_rows))

+-----------------+------+
|Weather_Condition| count|
+-----------------+------+
|            Clear|253070|
|             Fair| 96006|
|    Mostly Cloudy| 59929|
|    Partly Cloudy| 58523|
|         Overcast| 58472|
| Scattered Clouds| 29356|
|           Cloudy| 24063|
|             Haze| 21561|
|       Light Rain| 20421|
|             null| 12730|
|             Rain|  6438|
|              Fog|  4100|
|       Heavy Rain|  2246|
|            Smoke|  1921|
|     Fair / Windy|  1179|
+-----------------+------+

Rows removed: 15887


# Remove if distict values in category equals 1

In [18]:
df.persist()
tmp = [c for c in [*colCat] if df.select(countDistinct(c)).collect()[0][0] <= 1] 
df.unpersist()

[colCat.remove(c) for c in tmp]
print("Dropping columns with 1 class:", tmp)
logger.write2file("Dropping columns with 1 class", str(tmp))

Dropping columns with 1 class: ['Turning_Loop']


## Prepare Pipeline

In [19]:
imputer = Imputer(inputCols=colNum, outputCols=colNum)
imputer.setStrategy("median")

num_assembler = VectorAssembler(inputCols=colNum, outputCol="num_features")
scaler = MinMaxScaler(min=0.0, max=1.0, inputCol="num_features", outputCol="scaledFeatures")

indexers = [StringIndexer(inputCol = c, outputCol = c +'_IDX', handleInvalid='skip') for c in colCat]

encoder = OneHotEncoderEstimator(inputCols=[indexer.getOutputCol() for indexer in indexers], outputCols=["{0}_vec".format(indexer.getOutputCol()) for indexer in indexers])

## Preprocessing - Pipeline

In [21]:
# Categorical values

numPipeline = Pipeline(stages=[imputer, num_assembler, scaler])
catPipeline = Pipeline(stages=[*indexers, encoder])

pipeline = Pipeline(stages=[numPipeline, catPipeline])

preprocessed_df = pipeline.fit(df).transform(df)

In [22]:
preprocessed_df.select("scaledFeatures",*[c + "_IDX_vec" for c in colCat]).show(1,False)

+---------------------------------------------------------------------------------------------------------------------------+------------------+-------------------+--------------------+---------------------+-----------------+-------------+----------------------+-------------------------+---------------+-------------+----------------+----------------+----------------+---------------+---------------+------------------+---------------+-------------+-----------------------+----------------------+----------------------+----------------------+-------------------------+-----------------------------+
|scaledFeatures                                                                                                             |Start_Hour_IDX_vec|Start_Month_IDX_vec|Weather_Hour_IDX_vec|Weather_Month_IDX_vec|position_IDX_vec |Side_IDX_vec |Wind_Direction_IDX_vec|Weather_Condition_IDX_vec|Amenity_IDX_vec|Bump_IDX_vec |Crossing_IDX_vec|Give_Way_IDX_vec|Junction_IDX_vec|No_Exit_IDX_vec|Railway_IDX_vec

## Combine features

In [23]:
va2 = VectorAssembler(inputCols=["scaledFeatures", *[c + "_IDX_vec" for c in colCat]], outputCol="final_features")

df = va2.transform(preprocessed_df)
df.persist()
df = df.withColumn('label', col("Severity"))
df = df.withColumn('features', df.final_features).select("features","label")
df.unpersist()
#df_features.show(1, False)
#df = df_features

logger.write2fileModel(df, "df_features")

logger.write2file("Feature set size", str(df.count()) + "\n\n__Feature vector and label:__\n" + str(df.take(1)))
logger.write2file("Number of rows", str(df.count()))
print("Feature set size: ",df.count())

Saving into: models/CA_20200516131544/df_features
Saving through format 1
Feature set size:  635496


## Count categorical values

In [24]:
# Checking how many classes that can be used
#df.persist()
#tmp = [df.select(countDistinct(c).alias(c)).collect()[0] for c in [*colCat, "position"]] 
#df.unpersist()
print("Unique column values:", tmp)

#logger.write2file("Unique column values", str(tmp))

Unique column values: ['Turning_Loop']


# 3. Feature importance

## PCA
#In order to understand how much the variance affect the dataset we check with PCA. Try to get above 90%

In [25]:
k=377#k=250
pca = PCA(k=k, inputCol="features", outputCol="pca_features")
pca_model = pca.fit(df)
pca_df = pca_model.transform(df)


print("PCA - Feature Variance:","Top 50:\n" + str(pca_model.explainedVariance[:50]) + "\nNumber of items: "+str(k)+"\nSum of variance: "+ str(sum(pca_model.explainedVariance)))
logger.write2file("PCA - Feature variance", "Top 50:\n" + str(pca_model.explainedVariance[:50]) + "\nNumber of items: "+ str(k) +"\nSum of variance: "+ str(sum(pca_model.explainedVariance)))
logger.write2fileModel(pca_df, "pca_df")

PCA - Feature Variance: Top 50:
[0.09517194 0.04232475 0.03187734 0.02992612 0.02626072 0.02584163
 0.02481872 0.02431837 0.021698   0.0210608  0.01971418 0.019116
 0.01864003 0.01732142 0.01655527 0.01550796 0.01488046 0.01471253
 0.0138168  0.01375283 0.01310388 0.01297739 0.0125252  0.01215968
 0.01195906 0.0116045  0.01129448 0.01100461 0.01055348 0.01033876
 0.00993654 0.00958048 0.00940869 0.00864292 0.00853628 0.00799536
 0.00739191 0.00730536 0.0070218  0.00665225 0.00655656 0.00617407
 0.00588477 0.00584203 0.00550042 0.00536951 0.0048586  0.0047834
 0.00463187 0.00446751]
Number of items: 377
Sum of variance: 0.9928885632516312
Saving into: models/CA_20200516131544/pca_df
Saving through format 1


## ChiSqSelector
#Check top 100 which of the values in the feature vector

In [26]:
selector = ChiSqSelector(numTopFeatures=k, 
                         labelCol='label', 
                         featuresCol='features', 
                         outputCol="selectedFeatures",
                         selectorType='numTopFeatures', 
                         percentile=0.1, 
                         fpr=0.05, fdr=0.05, fwe=0.05)

chi_model = selector.fit(df)
chi_df = chi_model.transform(df)

logger.write2file("Top selected features according to ChiSqSelector", str(chi_model.selectedFeatures)+ "\nNumber of features: " + str(k) + "\nExample data:\n"+str(chi_df.take(5)))
logger.write2fileModel(chi_df, "chi_df")

print("Top selected features according to ChiSqSelector:", chi_model.selectedFeatures)
print("Transformed selected features:",chi_df.head().selectedFeatures)

Saving into: models/CA_20200516131544/chi_df
Saving through format 1
Top selected features according to ChiSqSelector: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 15, 16, 17, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 49, 51, 54, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 68, 69, 70, 71, 72, 73, 74, 75, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 90, 91, 92, 93, 94, 96, 98, 99, 100, 101, 102, 104, 105, 106, 107, 108, 109, 110, 113, 115, 116, 117, 118, 119, 120, 122, 125, 127, 128, 129, 131, 132, 133, 134, 135, 137, 138, 139, 141, 142, 144, 146, 147, 149, 150, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 181, 182, 183, 184, 185, 186, 188, 189, 190, 191, 192, 193, 194, 196, 198, 199, 201, 202, 203, 204, 205, 206, 207, 208, 210, 211, 212, 213, 214, 215, 217, 219, 221, 222, 223, 224, 226, 227, 229, 230, 231, 232, 233, 234, 235, 236, 239, 240, 241, 24

# 4. Machine learning

In [27]:
# Use the chi_df for training since it is reduced
#trainSet, testSet = df_features.randomSplit([0.8 ,0.2], 1)
trainSet, testSet = chi_df.randomSplit([0.8 ,0.2], 1)
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")

In [28]:
trainSet.show(2,False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                                                                                                                                                                                                                                                    |label|selectedFeatures                                                          

## Logistic regression

In [29]:
lr = LogisticRegression(labelCol="label", featuresCol="selectedFeatures")

paramGrid = ParamGridBuilder() \
                .addGrid(lr.regParam, [0.1,0.01]) \
                .addGrid(lr.maxIter, [10]) \
                .addGrid(lr.elasticNetParam, [0.6]) \
                .build()
try: 
    model, _ = evaluateModel(lr, paramGrid, "LR_Model", trainSet, testSet, evaluator=MulticlassClassificationEvaluator(),k=10, seed=None,logger=logger)
except Exception as e:
    logger.write2file("Error:", str(e))
    print(e)

Fitting model..
Predicting on testSet..
Evaluating predictions..
Saving into: models/CA_20200516131544/LR_Model
'function' object has no attribute 'format'
Saving through format 2
+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       2.0|    2|(528,[0,1,2,3,4,5...|
|       2.0|    2|(528,[0,1,2,3,4,5...|
|       3.0|    3|(528,[0,1,2,3,4,5...|
|       2.0|    2|(528,[0,1,2,3,4,5...|
|       2.0|    3|(528,[0,1,2,3,4,5...|
+----------+-----+--------------------+
only showing top 5 rows

Accuracy:  0.6883025028570805 
Parameters
 {Param(parent='LogisticRegression_f235d06c17ee', name='regParam', doc='regularization parameter (>= 0).'): 0.01, Param(parent='LogisticRegression_f235d06c17ee', name='maxIter', doc='max number of iterations (>= 0).'): 10, Param(parent='LogisticRegression_f235d06c17ee', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification report:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00        50
           2       0.73      0.94      0.82     85204
           3       0.70      0.31      0.43     40853
           4       0.00      0.00      0.00      1127

    accuracy                           0.73    127234
   macro avg       0.36      0.31      0.31    127234
weighted avg       0.71      0.73      0.69    127234

Confusion matrix:
[[    0    46     4     0]
 [    0 79844  5360     0]
 [    0 28184 12669     0]
 [    0   988   139     0]]


## Decision tree

In [30]:
dt = DecisionTreeClassifier(labelCol="label", featuresCol="selectedFeatures") 

paramGrid = ParamGridBuilder().build()

_, _ = evaluateModel(dt, paramGrid, "DT_Model", trainSet, testSet, evaluator=MulticlassClassificationEvaluator(),k=10, seed=None,logger=logger)

Fitting model..
Predicting on testSet..
Evaluating predictions..
Saving into: models/CA_20200516131544/DT_Model
'function' object has no attribute 'format'
Saving through format 2
+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       2.0|    2|(528,[0,1,2,3,4,5...|
|       2.0|    2|(528,[0,1,2,3,4,5...|
|       3.0|    3|(528,[0,1,2,3,4,5...|
|       2.0|    2|(528,[0,1,2,3,4,5...|
|       2.0|    3|(528,[0,1,2,3,4,5...|
+----------+-----+--------------------+
only showing top 5 rows

Accuracy:  0.5498715797285845 
Parameters
 {}
Classification report:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00        50
           2       0.67      1.00      0.80     85204
           3       0.77      0.02      0.04     40853
           4       1.00      0.00      0.00      1127

    accuracy                           0.67    127234
   macro avg       0.61      0.25      0

## Random forest

In [ ]:
rf = RandomForestClassifier(labelCol="label", featuresCol="selectedFeatures") #numTrees=10
paramGrid = ParamGridBuilder() \
                .addGrid(rf.numTrees,[15]) \
                .build()

_, _ = evaluateModel(rf, paramGrid, "RF_Model", trainSet, testSet, evaluator=MulticlassClassificationEvaluator(),k=10, seed=None,logger=logger)

Fitting model..


In [ ]:
logger.write2file("Program finished!", "")

In [ ]:
logger.write2file("New model folder created", \
               "Model finished: Yes" + \
               "\nFolder name: " + logger.timeSignature + \
               "\nState: "+ filter_state + \
               "\nLogs directory: " + logger.logs_dir +  \
               "\nFile: " + file + \
               "\nNote: " + model_Note, \
               logs_dir=model_indexing_dir \
              )
